# Merge station data from the 4D var and verification runs

In [61]:
import dask
dask.config.set(scheduler='processes')


In [22]:
import numpy as np 
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from datetime import datetime
from scipy.interpolate import PchipInterpolator

from glob import glob
from matplotlib import rcParams
from soda.dataio.roms.romsio import  get_depth
from soda.utils.myproj import MyProj
from mycurrents import oceanmooring as om
#rcParams['font.family'] = 'sans-serif'
#rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
#rcParams['font.serif'] = ['Bitstream Vera Sans']
rcParams["font.size"] = "12"
rcParams['axes.labelsize']='medium'

In [3]:
!ls /scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/


archive_fwd0_6241.nc	    fwd0_6293.nc      obs_6349.nc
archive_fwd0_6245.nc	    fwd0_6297.nc      obs_6353.nc
archive_fwd0_6249.nc	    fwd0_6301.nc      obs_6357.nc
archive_fwd0_6253.nc	    fwd0_6305.nc      qck_6241.nc
archive_fwd0_6257.nc	    fwd0_6309.nc      qck_6245.nc
archive_fwd0_6261.nc	    fwd0_6313.nc      qck_6249.nc
archive_fwd0_6265.nc	    fwd0_6317.nc      qck_6253.nc
archive_fwd0_6269.nc	    fwd0_6321.nc      qck_6257.nc
archive_fwd0_6273.nc	    fwd0_6325.nc      qck_6261.nc
archive_fwd0_6277.nc	    fwd0_6329.nc      qck_6265.nc
archive_fwd0_6281.nc	    fwd0_6333.nc      qck_6269.nc
archive_fwd0_6285.nc	    fwd0_6337.nc      qck_6273.nc
archive_fwd0_6289.nc	    fwd0_6341.nc      qck_6277.nc
archive_fwd0_6293.nc	    fwd0_6345.nc      qck_6281.nc
archive_fwd0_6297.nc	    fwd0_6349.nc      qck_6285.nc
archive_fwd0_6301.nc	    fwd0_6353.nc      qck_6289.nc
archive_fwd0_6305.nc	    fwd0_6357.nc      qck_6293.nc
archive_fwd0_6309.nc	    fwd1_6273.nc      qck_6297.nc
archive_fw

In [53]:
# Load a ROMS timeseries object
def get_roms_station_da(romsfile, romsvar, xyin, cycle=0, ncycles=3):
    dsroms = xr.open_dataset(romsfile)
    #print(dsroms)
    zroms = get_depth(dsroms.s_rho.values, dsroms.Cs_r.values, dsroms.hc.values, dsroms.h.values)# , \
    #         Vtransform=dsroms.Vtransform.values)
    xroms = dsroms.lon_rho.values
    yroms = dsroms.lat_rho.values
    
    nt = dsroms.ocean_time.size//ncycles

    # Get the point
    dist = np.abs( (xyin[0]-xroms) + 1j*(xyin[1]-yroms))
    idx = np.argwhere(dist==dist.min())[0,0]
    xyin, xroms[idx], yroms[idx], zroms[:,idx]
    
    t0 = cycle*nt
    t1 = t0 + nt

    data = dsroms[romsvar][t0:t1,idx,:].values
    #print(zroms[...].shape, data.shape)
    #print(zroms.ravel())

    #Fi = interp1d(zroms[...,idx].squeeze(), data.squeeze(), axis=1, fill_value='extrapolate')
    
    #return om.OceanMooring(dsroms.ocean_time.values[t0:t1], Fi(zin), zin)
    return om.OceanMooring(dsroms.ocean_time.values[t0:t1], data.squeeze(), zroms[...,idx].squeeze())

def get_roms_station(romsfile, romsvar, xyin):
    dsroms = xr.open_dataset(romsfile)
    #print(dsroms)
    zroms = get_depth(dsroms.s_rho.values, dsroms.Cs_r.values, dsroms.hc.values, dsroms.h.values)# , \
    #         Vtransform=dsroms.Vtransform.values)
    xroms = dsroms.lon_rho.values
    yroms = dsroms.lat_rho.values

    # Get the point
    dist = np.abs( (xyin[0]-xroms) + 1j*(xyin[1]-yroms))
    idx = np.argwhere(dist==dist.min())[0,0]
    #print(xyin, xroms[idx], yroms[idx], zroms[:,idx])

    data = dsroms[romsvar][:,idx,:]
    return om.OceanMooring(dsroms.ocean_time.values, data.squeeze(), zroms[...,idx].squeeze())
    #print(zroms[...].shape, data.shape)
    #print(zroms.ravel())
    
    #Fi = interp1d(zroms[...,idx].squeeze(), data.squeeze(), axis=1, fill_value='extrapolate')
    
    #return om.OceanMooring(dsroms.ocean_time.values, Fi(zin), zin)

In [54]:
# Add the site location
P = MyProj(None, utmzone=51,isnorth=False)

lon_prelude, lat_prelude = P.to_ll(534322.7, 8475878.4)

sites = pd.DataFrame(np.array([[lon_prelude, 123.3535, 123.2803, 123.346383,127.56335],\
            [lat_prelude, -13.7621, -13.8170, -13.75895, -9.8122]]).T,
            index=['Prelude','NP250','DWR','KP150_phs2','ITFTIS'], columns=['lon','lat'])


In [55]:
roms4dvarfiles = sorted(glob('/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_6*.nc'))
romsfiles = sorted(glob('/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_ver_6*.nc'))

In [56]:
romsfiles[0:2],roms4dvarfiles[0:2]

(['/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_ver_6253.nc',
  '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_ver_6257.nc'],
 ['/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_6241.nc',
  '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/sta_6253.nc'])

In [73]:
def load_station(loadfunc, roms4dvarfiles, sitename, varname):
    xyin = [sites['lon'][sitename],sites['lat'][sitename] ]

    ds = get_roms_station_da(roms4dvarfiles[0], varname, xyin)
    for ff in roms4dvarfiles[1:]:
        ds = ds.concat(get_roms_station_da(ff, varname, xyin))

    #ds.long_name = 'Water Temperature'
    #ds.units = 'degrees C'
    ds.X = xyin[0]
    ds.Y = xyin[1]
    ds.StationName = sitename
    return ds.to_xray()

sitename = 'NP250'
outfile = '../DATA/ROMS_4DVAR_station_{}.nc'.format(sitename)
T = load_station(get_roms_station_da, roms4dvarfiles, sitename,'temp')
u = load_station(get_roms_station_da, roms4dvarfiles, sitename,'u_eastward')
v = load_station(get_roms_station_da, roms4dvarfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

outfile = '../DATA/ROMS_NL_station_{}.nc'.format(sitename)
T = load_station(get_roms_station, romsfiles, sitename,'temp')
u = load_station(get_roms_station, romsfiles, sitename,'u_eastward')
v = load_station(get_roms_station, romsfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

../DATA/ROMS_4DVAR_station_NP250.nc
../DATA/ROMS_NL_station_NP250.nc


In [74]:
sitename = 'ITFTIS'
outfile = '../DATA/ROMS_4DVAR_station_{}.nc'.format(sitename)
T = load_station(get_roms_station_da, roms4dvarfiles, sitename,'temp')
u = load_station(get_roms_station_da, roms4dvarfiles, sitename,'u_eastward')
v = load_station(get_roms_station_da, roms4dvarfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

outfile = '../DATA/ROMS_NL_station_{}.nc'.format(sitename)
T = load_station(get_roms_station, romsfiles, sitename,'temp')
u = load_station(get_roms_station, romsfiles, sitename,'u_eastward')
v = load_station(get_roms_station, romsfiles, sitename,'v_northward')
xr.Dataset({'temp':T,'u_eastward':u,'v_northward':v}).to_netcdf(outfile)
print(outfile)

../DATA/ROMS_4DVAR_station_ITFTIS.nc
../DATA/ROMS_NL_station_ITFTIS.nc
